In [47]:
import forecast as fc
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder

In [48]:
df = pd.read_csv("files/pred_final.csv")
df

,obj_prg,obj_subprg,obj_key,Кодзадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,Статуспоэкспертизе,Экспертиза,date_report,season,Скорость,Кол-во рабочих,Генподрядчик,Генпроектировщик,Время на выполнение
0,0,0,022-0527,1,0,2022-01-14,2023-03-30,0,0,2023-01-17,1,0.051988,0.0,0.0,0.0,440
1,0,0,022-0354,1,0,2022-12-01,2023-07-27,0,0,2023-01-17,1,0.224265,0.0,0.0,0.0,238
2,0,1,022-0513,1,0,2023-01-23,2023-09-29,0,0,2023-01-17,1,0.117271,0.0,0.0,0.0,249
3,0,1,020-0684,1,0,2020-11-03,2022-02-01,0,0,2023-01-17,4,0.000000,0.0,2.0,1.0,455
4,0,1,019-0589,1,0,2020-11-03,2022-05-16,0,0,2023-01-17,4,0.000000,0.0,2.0,1.0,559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35196,1,2,019-0477,7.4,93,2020-04-28,2023-05-31,0,0,2023-05-29,2,0.082447,5.0,12.0,16.0,1128
35197,0,0,020-0710,8,0,2023-08-31,2024-04-10,0,0,2023-05-29,3,0.000000,156.0,5.0,8.0,223
35198,0,0,019-0675,8,0,2023-09-01,2024-04-11,0,0,2023-05-29,4,0.000000,139.0,22.0,33.0,223
35199,0,1,020-0712,8,0,2023-06-19,2024-01-26,0,0,2023-05-29,2,0.000000,53.0,5.0,8.0,221


In [49]:
test_df = df.sort_values(['obj_key', 'Кодзадачи', 'ПроцентЗавершенияЗадачи', 'date_report'])
test_df

,obj_prg,obj_subprg,obj_key,Кодзадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,Статуспоэкспертизе,Экспертиза,date_report,season,Скорость,Кол-во рабочих,Генподрядчик,Генпроектировщик,Время на выполнение
8,0,1,017-0520,1,100,2018-11-01,2022-02-15,0,0,2023-01-17,4,0.083195,0.0,5.0,8.0,1202
1699,0,1,017-0520,1,100,2018-11-01,2022-02-15,0,0,2023-01-23,4,0.083195,0.0,5.0,8.0,1202
3432,0,1,017-0520,1,100,2018-11-01,2022-02-15,0,0,2023-02-06,4,0.083195,0.0,5.0,8.0,1202
5219,0,1,017-0520,1,100,2018-11-01,2022-02-15,0,0,2023-02-13,4,0.083195,0.0,5.0,8.0,1202
7030,0,1,017-0520,1,100,2018-11-01,2022-02-15,0,0,2023-02-20,4,0.083195,0.0,5.0,8.0,1202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33242,1,3,022-0631,7.4,0,2024-10-14,2024-11-11,0,0,2023-05-22,4,0.000000,0.0,2.0,1.0,28
35153,1,3,022-0631,7.4,0,2024-10-14,2024-11-11,0,0,2023-05-29,4,0.000000,0.0,2.0,1.0,28
1665,1,3,022-0631,8,0,2024-12-02,2025-07-11,0,0,2023-01-17,1,0.000000,0.0,2.0,0.0,221
3393,1,3,022-0631,8,0,2024-12-02,2025-07-11,0,0,2023-01-23,1,0.000000,0.0,2.0,0.0,221


In [50]:
lk = sk.preprocessing.LabelEncoder()
lk.fit(test_df["Кодзадачи"])
test_df["Кодзадачи"] = lk.transform(test_df["Кодзадачи"])
test_df

,obj_prg,obj_subprg,obj_key,Кодзадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,Статуспоэкспертизе,Экспертиза,date_report,season,Скорость,Кол-во рабочих,Генподрядчик,Генпроектировщик,Время на выполнение
8,0,1,017-0520,0,100,2018-11-01,2022-02-15,0,0,2023-01-17,4,0.083195,0.0,5.0,8.0,1202
1699,0,1,017-0520,0,100,2018-11-01,2022-02-15,0,0,2023-01-23,4,0.083195,0.0,5.0,8.0,1202
3432,0,1,017-0520,0,100,2018-11-01,2022-02-15,0,0,2023-02-06,4,0.083195,0.0,5.0,8.0,1202
5219,0,1,017-0520,0,100,2018-11-01,2022-02-15,0,0,2023-02-13,4,0.083195,0.0,5.0,8.0,1202
7030,0,1,017-0520,0,100,2018-11-01,2022-02-15,0,0,2023-02-20,4,0.083195,0.0,5.0,8.0,1202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33242,1,3,022-0631,26,0,2024-10-14,2024-11-11,0,0,2023-05-22,4,0.000000,0.0,2.0,1.0,28
35153,1,3,022-0631,26,0,2024-10-14,2024-11-11,0,0,2023-05-29,4,0.000000,0.0,2.0,1.0,28
1665,1,3,022-0631,27,0,2024-12-02,2025-07-11,0,0,2023-01-17,1,0.000000,0.0,2.0,0.0,221
3393,1,3,022-0631,27,0,2024-12-02,2025-07-11,0,0,2023-01-23,1,0.000000,0.0,2.0,0.0,221


In [51]:
test_df['ДатаНачалаЗадачи'] =  pd.to_datetime(test_df['ДатаНачалаЗадачи']).apply(lambda x: x.toordinal())
test_df['ДатаОкончанияЗадачи'] =  pd.to_datetime(test_df['ДатаОкончанияЗадачи']).apply(lambda x: x.toordinal())
test_df

,obj_prg,obj_subprg,obj_key,Кодзадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,Статуспоэкспертизе,Экспертиза,date_report,season,Скорость,Кол-во рабочих,Генподрядчик,Генпроектировщик,Время на выполнение
8,0,1,017-0520,0,100,736999,738201,0,0,2023-01-17,4,0.083195,0.0,5.0,8.0,1202
1699,0,1,017-0520,0,100,736999,738201,0,0,2023-01-23,4,0.083195,0.0,5.0,8.0,1202
3432,0,1,017-0520,0,100,736999,738201,0,0,2023-02-06,4,0.083195,0.0,5.0,8.0,1202
5219,0,1,017-0520,0,100,736999,738201,0,0,2023-02-13,4,0.083195,0.0,5.0,8.0,1202
7030,0,1,017-0520,0,100,736999,738201,0,0,2023-02-20,4,0.083195,0.0,5.0,8.0,1202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33242,1,3,022-0631,26,0,739173,739201,0,0,2023-05-22,4,0.000000,0.0,2.0,1.0,28
35153,1,3,022-0631,26,0,739173,739201,0,0,2023-05-29,4,0.000000,0.0,2.0,1.0,28
1665,1,3,022-0631,27,0,739222,739443,0,0,2023-01-17,1,0.000000,0.0,2.0,0.0,221
3393,1,3,022-0631,27,0,739222,739443,0,0,2023-01-23,1,0.000000,0.0,2.0,0.0,221


In [52]:
x = test_df.drop(['Время на выполнение', 'ДатаОкончанияЗадачи', 'date_report', 'Время на выполнение', 'obj_key'],
                 axis=1)
y = test_df['Время на выполнение']
y

8        1202
1699     1202
3432     1202
5219     1202
7030     1202
         ... 
33242      28
35153      28
1665      221
3393      221
5191      221
Name: Время на выполнение, Length: 35201, dtype: int64

In [53]:
split_index = int(len(test_df) * 0.8)
x_train = x[:split_index]
x_test = x[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

In [54]:
one_hot_encoder = make_column_transformer(
                      (
                          OneHotEncoder(sparse_output=False, drop='if_binary'),
                          make_column_selector(dtype_exclude=np.number),
                      ),
                      remainder="passthrough",
                      verbose_feature_names_out=False,
                  ).set_output(transform="pandas")

In [55]:
forecaster = ForecasterAutoreg(
                 regressor = CatBoostRegressor(
                                        random_state=123,
                                        silent=True,
                                        allow_writing_files=False
                                    ),
                 lags             = 10,
                 transformer_exog = one_hot_encoder
             )

In [58]:
param_grid = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [48, 72]

results_grid = grid_search_forecaster(
                   forecaster         = forecaster,
                   y                  = y_train,
                   exog               = x_train,
                   param_grid         = param_grid,
                   lags_grid          = lags_grid,
                   steps              = 36,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(x_train)-1,
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 24.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/12 [00:00<?, ?it/s]

C:\Users\I\PycharmProjects\dom_stroy\venv\lib\site-packages\skforecast\utils\utils.py:801: UserWarning: `y` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(
C:\Users\I\PycharmProjects\dom_stroy\venv\lib\site-packages\skforecast\utils\utils.py:928: UserWarning: `exog` has no DatetimeIndex nor RangeIndex index. Index is overwritten with a RangeIndex.
  warnings.warn(


CatBoostError: Length of label=28111 and length of data=33636 is different.